Using the Bedtools function to compare C. elegans expression data from fasta files.

First, we find out some information about what data the fasta file contains. 

In [1]:
# Retrieving how many chromosomes are in C. elegans and what their names are from the ce10.fa file 

#Getting the names of each chromosome by searching for the ">" header delimiter with grep, 
    #and extracting those values with cut -f 1 from the first column
grep '>' ce10.fa | cut -f 1 

# Adding to previous code where we got the chromosome names to count the number of those chromosomes with 'wc -l'
grep '>' ce10.fa | cut -f 1 | wc -l 

>chrI
>chrII
>chrIII
>chrIV
>chrV
>chrX
>chrM
7


In [2]:
# Finding how many unique features the 'ce10.refGene.gtf' file has and what their names are 

# Using cut to extract the third column of the file where the features are, 
    #sorting by feature using sort, 
    #isolating the unique features with uniq to display each unique feature
cut -f 3 ce10.refGene.gtf | sort | uniq 

# Using the code above and adding wc -l to count the number of features
cut -f 3 ce10.refGene.gtf | sort | uniq | wc -l

3UTR
5UTR
CDS
exon
start_codon
stop_codon
transcript
7


In [3]:
# Creating a new file 'ce10_UTR.txt' to hold all of the annotated UTRs (5' and 3') from ce10.refGene.gtf
touch ce10_UTR.txt 

# extracting all rows that contain "UTR" and moving them to the ce10_UTR.txt file
grep 'UTR' ce10.refGene.gtf > ce10_UTR.txt 

In [4]:
# Finding how many UTRs are annotated, then how many are 5' UTRs 

# Getting the number of rows in the ce10_UTR.txt file which is the total annotated UTRs 
wc -l ce10_UTR.txt 

# Extracting the rows with 5UTR as a feature and counting those with wc -l
grep '5UTR' ce10_UTR.txt | wc -l 

51318 ce10_UTR.txt
24874


In [6]:
# Renaming the .tab file to a .bed extension 
mv ce10_tRNAgenes.tab ce10_tRNAgenes.bed

In [7]:
# Creating a new file “ce10_mammalian_limiting_tRNAgenes.bed” that stores the 
    #tRNA genes that encode for the 4 limiting amono acids
# The four limiting aa's are lysine (leu), threonine (thr), methionine (met), and tryptophan (trp)

touch ce10_mammalian_limiting_tRNAgenes.bed
grep 'Leu' ce10_tRNAgenes.bed > ce10_mammalian_limiting_tRNAgenes.bed
grep 'Thr' ce10_tRNAgenes.bed >> ce10_mammalian_limiting_tRNAgenes.bed
grep 'Met' ce10_tRNAgenes.bed >> ce10_mammalian_limiting_tRNAgenes.bed
grep 'Trp' ce10_tRNAgenes.bed >> ce10_mammalian_limiting_tRNAgenes.bed

In [8]:
# Finding the number of tRNA genes that encode for the mammalian limiting amino acids 

# Using wc -l to count the rows in the file which represent the number of genes
wc -l ce10_mammalian_limiting_tRNAgenes.bed 

104 ce10_mammalian_limiting_tRNAgenes.bed


Now, we load and use the bedtools module to compare and find overlap in C. elegans tRNA genes to C. elegans transcribed genes. 

In [9]:
# Searching for the bedtools module 
module avail bedtools


--------------------------- /share/apps/modulefiles ----------------------------
   bedtools/intel/2.29.2


In [11]:
# Printing the contents of the sbatch file that loads and properly executes the bedtools function
cat tRNAgenes_bedtools.sbatch

#!/bin/bash
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=1
#SBATCH --time=1:00:00
#SBATCH --mem=2GB
#SBATCH --job-name=ce10_tRNA_bedtools

# Purging and loading the bedtools module 
module purge 
module load bedtools/intel/2.29.2

# using 'bedtools intersect -u' to compare ce10_tRNAgenes.bed to ce10_pol2transcribedgenes.bed and put where the two files overlap into ce10_tRNAgenes_Pol2_intersect.bed
bedtools intersect -u -a ce10_tRNAgenes.bed -b ce10_pol2transcribedgenes.bed > ce10_tRNAgenes_Pol2_intersect.bed


In [12]:
# Running the sbatch file 
sbatch tRNAgenes_bedtools.sbatch 

Submitted batch job 43390992


In [13]:
# Finding how many tRNA genes overlap with an RNA Pol II gene  
wc -l ce10_tRNAgenes_Pol2_intersect.bed

65 ce10_tRNAgenes_Pol2_intersect.bed


In [14]:
# Finding how many of these genes are tRNA that encode for the mammalian limiting amino acids
# lysine (leu), threonine (thr), methionine (met), and tryptophan (trp)

# Using grep to isolate the rows with the limiting amino acid codes,
    #then piping to wc -l to find how many
grep 'Leu\|Thr\|Met\|Trp' ce10_tRNAgenes_Pol2_intersect.bed | wc -l 

9
